# Calculating Trapped Rainwater using Pandas
When it rains, water naturally accumulates in low-lying areas, forming small pools between elevated terrains. This phenomenon is commonly seen in cities with poor drainage systems, uneven roads, or natural landscapes with hills and valleys. But what if we wanted to quantify exactly how much water gets trapped in such a terrain?

This problem, commonly referred to as the "Trapping Rain Water Problem," is a classic coding challenge in algorithm design. It requires us to analyze a 1D elevation map, determine where water can be stored, and compute the total amount of trapped rainwater.

In this blog post, I’ll walk you through a Python-based approach using Pandas to efficiently compute trapped rainwater in a given elevation profile. By leveraging cumulative maximum calculations, we can determine the trapped water at each position and sum it up—all with minimal computational overhead.

We’ll explore the step-by-step logic behind this solution, from understanding how water accumulates in terrain to implementing an optimized approach using Pandas. By the end, you’ll have a solid grasp of how to tackle this problem in an efficient and elegant way.

Let’s dive in!

## Problem Statement

Given a table `Heights` representing the elevation map, we need to determine how much rainwater can be trapped between the bars in the landscape, considering that each bar has a width of 1 unit.

### Table Schema

**Heights Table:**

| Column Name | Type |
|-------------|------|
| id          | int  |
| height      | int  |

- `id` is the primary key (column with unique values) for this table.
- The `id` values are in sequential order.
- Each row of this table contains an `id` and `height`, representing the elevation at that position.

---

## Example

### **Input**: Heights Table

| id  | height |
|-----|--------|
| 1   | 0      |
| 2   | 1      |
| 3   | 0      |
| 4   | 2      |
| 5   | 1      |
| 6   | 0      |
| 7   | 1      |
| 8   | 3      |
| 9   | 2      |
| 10  | 1      |
| 11  | 2      |
| 12  | 1      |

### **Output**:

| total_trapped_water |
|---------------------|
| 6                   |





In [26]:
import pandas as pd

data = [[1, 0], [2, 1],
        [3, 0], [4, 2],
        [5, 1], [6, 0],
        [7, 1], [8, 3],
        [9, 2], [10, 1],
        [11, 2], [12, 1]]
heights = pd.DataFrame(data,
                       columns=['id', 'height']).astype({
                       'id':'Int64', 'height':'Int64'})
display(heights)

,id,height
0,1,0
1,2,1
2,3,0
3,4,2
4,5,1
5,6,0
6,7,1
7,8,3
8,9,2
9,10,1


Step 1: Sorting the Data by 'id'
- This sorts the DataFrame by the 'id' column in ascending order.

In [27]:
heights= heights.sort_values(by='id')
display(heights)

,id,height
0,1,0
1,2,1
2,3,0
3,4,2
4,5,1
5,6,0
6,7,1
7,8,3
8,9,2
9,10,1


Step 2: Computing the Left Maximum Height at Each Position
- cummax() computes the cumulative maximum from left to right.
- This means at each position, it stores the maximum height encountered so far.

In [28]:
heights['left_max'] = heights['height'].cummax()
display(heights)

,id,height,left_max
0,1,0,0
1,2,1,1
2,3,0,1
3,4,2,2
4,5,1,2
5,6,0,2
6,7,1,2
7,8,3,3
8,9,2,3
9,10,1,3


Step 3: Computing the Right Maximum Height at Each Position
- [::-1] reverses the DataFrame to compute the cumulative max from right to left.
- .cummax() computes the maximum height encountered from the right.
- The array is then reversed back to match the original order.

In [29]:
heights['right_max'] = heights['height'][::-1].cummax()[::-1].values
display(heights)

,id,height,left_max,right_max
0,1,0,0,3
1,2,1,1,3
2,3,0,1,3
3,4,2,2,3
4,5,1,2,3
5,6,0,2,3
6,7,1,2,3
7,8,3,3,3
8,9,2,3,2
9,10,1,3,2


Step 4: Computing the Minimum of Left and Right Maximums
- This finds the minimum between the left max and right max for each position.
- This represents the maximum water level that can be held at each position.

In [30]:
heights['min_of_right_left'] = heights[['left_max','right_max']].min(axis=1)
display(heights)

,id,height,left_max,right_max,min_of_right_left
0,1,0,0,3,0
1,2,1,1,3,1
2,3,0,1,3,1
3,4,2,2,3,2
4,5,1,2,3,2
5,6,0,2,3,2
6,7,1,2,3,2
7,8,3,3,3,3
8,9,2,3,2,2
9,10,1,3,2,2


Step 5: Computing the Trapped Water at Each Position
- Since height is always less than or equal to min_of_right_left, abs() isn't needed.

In [31]:
heights['trapped'] = abs(heights['height'] - heights['min_of_right_left'])
display(heights)

,id,height,left_max,right_max,min_of_right_left,trapped
0,1,0,0,3,0,0
1,2,1,1,3,1,0
2,3,0,1,3,1,1
3,4,2,2,3,2,0
4,5,1,2,3,2,1
5,6,0,2,3,2,2
6,7,1,2,3,2,1
7,8,3,3,3,3,0
8,9,2,3,2,2,0
9,10,1,3,2,2,1


Step 6: Computing the Total Trapped Water
- heights['trapped'].sum() calculates the total trapped water across all positions.
- The result is stored in a new DataFrame df with a single column 'total_trapped_water'.

In [32]:
df = pd.DataFrame({'total_trapped_water':[heights['trapped'].sum()]})
display(df)

,total_trapped_water
0,6


Reference: [1] https://leetcode.com/problems/calculate-trapping-rain-water/description/